In [ ]:
import pandas as pd
from shopifyapi import ShopifyApp
import os
from dotenv import load_dotenv
from converter import to_handle

In [ ]:
load_dotenv()

# Support Functions

In [ ]:
def adding_upc(df):
    try:
        upc_df = pd.read_csv('hobbytron UPC 2024 added.csv')
        upc_df.fillna('', inplace=True)
        for index in range(len(df)):
            if pd.notna(df.loc[index, 'Variant SKU']):
                df.loc[index, 'Variant Barcode'] = upc_df[upc_df['Available'] == ''].iloc[0, 0].copy()
                df.loc[index, 'Google Shopping / MPN'] = upc_df[upc_df['Available'] == ''].iloc[0, 0].copy()
                index = upc_df.index[upc_df['Available'] == ''][0]
                upc_df.iloc[index, 1] = 'Used'
            else:
                df.loc[index, 'Variant Barcode'] = ''
                df.loc[index, 'Google Shopping / MPN'] = ''
        upc_df.to_csv('hobbytron UPC 2024 added.csv', index=False)

        return df
    
    except IndexError:
        print('UPC are not available!')

In [ ]:
def adding_tags(title, tags):
    try:
        if pd.notna(title):
            if pd.notna(tags):
                result = tags + ', toys'
            else:
                result = 'toys'
        else:
            result = None
        
        return result
    except Exception as e:
        print(e)
            

# Get Collection

In [ ]:
df = pd.read_csv('product_data/products_output.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Collection Rule'] = df['Tags'].str.split(',')

In [ ]:
parent_product_df = df[~pd.isna(df['Title'])]

In [ ]:
exploded_parent_product_df = parent_product_df.explode('Collection Rule', ignore_index=True)

In [ ]:
exploded_parent_product_df['Collection Rule'] = exploded_parent_product_df['Collection Rule'].str.strip()

In [ ]:
exploded_parent_product_df['Collection Name'] = exploded_parent_product_df['Collection Rule'].str.title()

In [ ]:
collection_df = exploded_parent_product_df[['Collection Name', 'Collection Rule']]

In [ ]:
collection_df.dropna(inplace=True, ignore_index=True)

In [ ]:
collection_df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
collection_df

# Shopify

In [ ]:
shopify = ShopifyApp(store_name=os.getenv('STORE_NAME'), access_token=os.getenv('ACCESS_TOKEN'))
client = shopify.create_session()

In [ ]:
for index in range(len(collection_df)):
    print(collection_df.iloc[index, 0])

In [ ]:
has_next_page = True
cursor = None
results = list()
while has_next_page:
    response = shopify.get_collections(client, cursor=cursor)
    records = response['data']['collections']['nodes']
    results.extend(records)
    has_next_page = response['data']['collections']['pageInfo']['hasNextPage']
    cursor = response['data']['collections']['pageInfo']['endCursor']
results_df = pd.DataFrame.from_records(results)
results_df.to_csv('collection_list.csv', index=False)

In [ ]:
shopify_col_df = pd.read_csv('collection_list.csv')

In [ ]:
collection_df['Handle'] = collection_df.apply(lambda x: to_handle(title=x['Collection Name'], alt_title=''), axis=1)

In [ ]:
collection_df = collection_df.merge(shopify_col_df, how='left', left_on='Handle', right_on='handle')

In [ ]:
collection_df

In [ ]:
collection_df[~pd.isna(collection_df['id'])]

In [ ]:
collection_df[pd.isna(collection_df['id'])]

# Adding UPC

In [ ]:
products_df = pd.read_csv('product_data/products_output.csv')

In [ ]:
products_df.head()[['Handle','Variant SKU']]

In [ ]:
products_df = adding_upc(products_df)

In [ ]:
products_df.to_csv('product_data/products_with_upc.csv', index=False)

# Adding Tags

In [ ]:
products_df = pd.read_csv('product_data/products_with_upc.csv')

In [ ]:
products_df['Tags'] = products_df.apply(lambda x: adding_tags(x['Title'], x['Tags']), axis=1)

In [ ]:
products_df[['Handle','Title','Tags', 'Vendor']]

# change Vendor

In [ ]:
products_df['Vendor']

In [ ]:
products_df['Vendor'] = products_df['Vendor'].apply(lambda x: 'Htrn71' if x == 'Hobbytron' else None)

In [ ]:
products_df['Vendor']

In [ ]:
products_df.to_csv('products_with_upc_upd_vendor_tags_rev1.csv', index=False)